# Ideal gas equation of state using grand canonical ensemble transition-matrix Monte Carlo

In this example, the ideal gas equation of state is obtained as a test of the flat histogram method.

In [1]:
params={"cubic_box_length": 8, "beta": 1./1.2, "mu": -3}
script="""
MonteCarlo
Configuration cubic_box_length {cubic_box_length} particle_type /feasst/forcefield/atom.fstprt
Potential Model IdealGas
ThermoParams beta {beta} chemical_potential {mu}
FlatHistogram Macrostate MacrostateNumParticles width 1 min 0 max 50 \
              Bias TransitionMatrix min_sweeps 100
TrialTransfer particle_type 0
CriteriaUpdater trials_per 1e5
CriteriaWriter trials_per 1e5 file_name id_fh.txt
Run until_criteria_complete true
""".format(**params)

def run(script):
    with open('script.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("~/feasst/build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# FEASST version: v0.19.0-59-gb2fd5186f4-dirty-hwh/newtutorials
Configuration cubic_box_length 8 particle_type /home/hwh/feasst/forcefield/atom.fstprt  
Potential Model IdealGas  
ThermoParams beta 0.8333333333333334 chemical_potential -3  
FlatHistogram Bias TransitionMatrix Macrostate MacrostateNumParticles max 50 min 0 min_sweeps 100 width 1  
TrialTransfer particle_type 0  
CriteriaUpdater trials_per 1e5  
CriteriaWriter file_name id_fh.txt trials_per 1e5  
Run until_criteria_complete true  
# initializing random number generator with seed: 1652465998
 
 exit: 0


Check the ideal gas relationship, $\beta P = \rho$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

fh=pd.read_csv('id_fh.txt', comment="#")
#print(fh)
print('N betaPV difference')
for delta_conjugate in np.arange(-5, 5, 0.1):
    fh['ln_prob_rw'] = fh['ln_prob'] + fh['state']*delta_conjugate - fh['ln_prob'].min()  # avoid negative log
    fh['ln_prob_rw'] -= np.log(sum(np.exp(fh['ln_prob_rw'])))   # renormalize
    if fh['ln_prob_rw'].values[-1] < -6:
        #plt.plot(fh['ln_prob_rw'])
        N = (np.exp(fh["ln_prob_rw"]) * fh["state"]).sum()
        betaPV = -fh["ln_prob_rw"][0] - np.log(np.exp(fh["ln_prob_rw"]).sum())
        print(N, betaPV, N-betaPV)
        assert(np.abs(1 - betaPV/N) < 1e-2)

N betaPV difference
0.281615437360333 0.28143886331769313 0.00017657404263987653
0.3112721290573395 0.3110584748658264 0.00021365419151309073
0.34405568678745374 0.3437974829119714 0.0002582038754823657
0.38029641695368716 0.37998481251059724 0.00031160444308991897
0.4203595816256132 0.41998413789932987 0.0003754437262833421
0.46464908608091554 0.46419755942857654 0.000451526652338996
0.5136115446272471 0.513069667880601 0.000541876746646075
0.5677407591737768 0.5670920357124472 0.0006487234613296033
0.6275826471274961 0.626808178308256 0.0007744688192401483
0.69374065753872 0.6928190320982597 0.0009216254404603719
0.7668817173956256 0.7657890004354689 0.00109271696015667
0.847742754248729 0.846452622510358 0.0012901317383710031
0.9371378480250632 0.9356219255100562 0.0015159225150069844
1.0359660755637645 1.0341945260068717 0.0017715495568928485
1.1452201281579077 1.1431625536913854 0.0020575744665223272
1.2659958075469175 1.263622479772529 0.0023733277743884784
1.3995025412074162 1.3

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!